In [36]:
# from google.colab import drive
# drive.mount('/content/drive')

### 0. Загрузка данных

In [37]:
!pip install mne
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv

--2025-12-23 00:45:10--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5381138 (5.1M) [text/plain]
Saving to: ‘train.csv.3’

train.csv.3         100%[===================>]   5.13M  --.-KB/s    in 0.08s   

2025-12-23 00:45:11 (61.4 MB/s) - ‘train.csv.3’ saved [5381138/5381138]

--2025-12-23 00:45:11--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3567877 (3.4M

In [38]:
!pip install lightautoml

In [39]:
import pandas as pd
import numpy as np
import mne

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

from src.dataset import get_target, calc_features

delimiter = "\n" + "-"*10

In [40]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,time,condition,epoch,C3,Cz,C4,Fp1,Fp2,F7,F3,...,F8,T7,T8,P7,P3,Pz,P4,P8,O1,O2
0,0,1,0,-5.885714,-2.533107,9.866895,7.962973,5.694433,23.638605,27.899784,...,23.588723,12.178548,0.685809,-4.887397,-10.646985,-14.735646,-8.729323,-0.562578,-17.055458,-3.616732
1,1,1,0,-7.999715,-16.916729,-11.924855,17.955477,8.526994,56.635981,28.508435,...,1.045533,14.656061,-4.119778,-4.632381,-17.980657,-23.456960,-12.960684,-9.639784,-20.233549,-1.229811
2,2,1,0,-6.727283,-15.979567,-11.114195,17.183478,4.497028,43.914130,10.079754,...,-14.741630,14.793562,-6.624813,-3.402757,-10.269473,-18.736144,-3.579046,-0.045658,-14.089755,2.453398
3,3,1,0,6.819390,-0.204905,10.090124,20.265222,7.843006,36.250611,13.291199,...,-7.135541,21.723418,-2.276825,2.066859,4.325365,-2.803322,8.835114,13.878945,-5.772410,7.657873
4,4,1,0,13.129486,-5.817193,5.040633,19.462210,9.634234,42.311729,20.641012,...,0.015602,19.703190,3.739076,2.714350,3.251047,-3.631448,3.212956,4.635574,-5.045448,5.086024


In [41]:
non_eeg = ['time', 'condition', 'epoch', 'target']
channels = [ch for ch in df_train.columns if ch not in non_eeg]

In [42]:
X = get_target(df_train)
X = X.merge(calc_features(df_train, channels), on='epoch')
y = X['condition'].apply(lambda x: 0 if x == 1 else 1)

X = X.drop(['epoch', 'condition'], axis=1)
y = y.astype(int)
y.name = 'target'
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

## 3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных

[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения

[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

In [43]:
# Cтратегии разделения данных:
# В соответствием с выводами из файла EDA (part 1) делаем корректный train-test split по данным, предварительно подготовленным по эпохам
npr = np.random.RandomState(42)
epoch_list = X.index
train_idx, test_idx = train_test_split(epoch_list, test_size=0.33, random_state=42) ### делаем разбиение по эпохам как независимым объектам, чтобы избежать утечки между зависимыми наблюдениями, поскольку эпохи коррелированы внутри себя

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]


In [44]:
# Подготовим данные для конфигураций
roles = {'target': 'target'}
task = Task('binary')

train_LAMA = X_train.copy()
train_LAMA['target'] = y_train.values

test_LAMA = X_test.copy()
test_LAMA['target'] = y_test.values

#### Конфигурация 1 - Классический LAMA (default)
Стандартная конфигурация - подбор лучших моделей (LGB, CatBoost и др.) с автотюнингом. Используем пресеты "по умолчанию".

In [45]:
automl1 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
)
oof_pred1 = automl1.fit_predict(train_LAMA, roles=roles, verbose=2)
test_pred1 = automl1.predict(test_LAMA)

auc1 = roc_auc_score(y_test, test_pred1.data[:, 0])
f1_1 = f1_score(y_test, test_pred1.data[:, 0] > 0.5)
print(f"[LAMA Default] ROC-AUC: {auc1:.4f} F1: {f1_1:.4f}")

[00:45:31] Stdout logging level is INFO2.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO2.


[00:45:31] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


[00:45:31] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



...

[00:46:46] Automl preset training completed in 75.35 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 75.35 seconds



[00:46:46] Model description:
Final prediction for new objects (level 0) = 
	 0.61633 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07663 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.30704 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.61633 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07663 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.30704 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



[LAMA Default] ROC-AUC: 0.7217 F1: 0.6557


### Конфигурация 2 — FAST, только lgbm
Используем только быстрый LightGBM.

In [46]:
automl2 = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb']]
    }
)
oof_pred2 = automl2.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred2 = automl2.predict(test_LAMA)

auc2 = roc_auc_score(y_test, test_pred2.data[:, 0])
f1_2 = f1_score(y_test, test_pred2.data[:, 0] > 0.5)
print(f'[LAMA FAST (lgb only)] ROC-AUC: {auc2:.4f}  F1: {f1_2:.4f}')


[00:46:46] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[00:46:46] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[00:46:46] Start automl preset with listed constraints:


...

[00:46:47] Automl preset training completed in 0.53 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 0.53 seconds



[00:46:47] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) 



[LAMA FAST (lgb only)] ROC-AUC: 0.7138  F1: 0.6552


### Конфигурация 3 - расширенный набор моделей и мощный автотюнинг

Включаем сразу несколько ансамблей (XGB, LGBM с тюнингом, CatBoost с тюнингом, MLP нейросеть) + используем продвинутую кросс-валидацию, автотюнинг с разными алгоритмами

In [47]:
automl3 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['xgb', 'lgb_tuned', 'cb_tuned'], ['mlp']],  # Много моделей
    },
    reader_params={
        'n_jobs': 2,
        'cv': 5,
        'random_state': 84,
        'advanced_roles': True,
    },
    tuning_params={'max_tuning_iter': "auto", 'max_tuning_time': 300},
    selection_params={'mode': 1}
)

oof_pred3 = automl3.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred3 = automl3.predict(test_LAMA)

auc3 = roc_auc_score(y_test, test_pred3.data[:, 0])
f1_3 = f1_score(y_test, test_pred3.data[:, 0] > 0.5)
print(f'[LAMA Advanced] ROC-AUC: {auc3:.4f}  F1: {f1_3:.4f}')


[00:46:47] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[00:46:47] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[00:46:47] Start automl preset with listed constraints:


...

[00:47:23] Automl preset training completed in 36.15 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 36.15 seconds



[00:47:23] Model description:
Models on level 0:
	 5 averaged models Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM
	 5 averaged models Lvl_0_Pipe_0_Mod_1_Tuned_CatBoost
	 5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost

Final prediction for new objects (level 1) = 
	 1.00000 * (5 averaged models Lvl_1_Pipe_0_Mod_0_TorchNN_mlp_0) 



INFO:lightautoml.automl.presets.base:Model description:
Models on level 0:
	 5 averaged models Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM
	 5 averaged models Lvl_0_Pipe_0_Mod_1_Tuned_CatBoost
	 5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost

Final prediction for new objects (level 1) = 
	 1.00000 * (5 averaged models Lvl_1_Pipe_0_Mod_0_TorchNN_mlp_0) 



[LAMA Advanced] ROC-AUC: 0.5000  F1: 0.0000


### конфигурация 4 - с feature selection и ограниченным набором моделей
Только LGBM (c hyperparameter tuning) и линейная регрессия; включён feature selection (оставляем 80% лучших признаков); гиперпараметры подбираются на holdout-валидации.


In [48]:
automl4 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb_tuned', 'linear_l2']]  # Только LGBM + Linear
    },
    selection_params={
        'select_algos': ['gbm', 'linear_l2'],
        'selection_feats_rate': 0.8,
        'max_features_cnt_in_result': None,
    },
    tuning_params={
        'fit_on_holdout': True,
        'max_tuning_iter': 50,
        'max_tuning_time': 200,
    }
)

oof_pred4 = automl4.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred4 = automl4.predict(test_LAMA)

auc4 = roc_auc_score(y_test, test_pred4.data[:, 0])
f1_4 = f1_score(y_test, test_pred4.data[:, 0] > 0.5)
print(f'[LAMA FS+LGBM/Linear] ROC-AUC: {auc4:.4f}  F1: {f1_4:.4f}')


[00:47:25] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[00:47:25] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[00:47:25] Start automl preset with listed constraints:


...

[00:47:39] Automl preset training completed in 14.12 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 14.12 seconds



[00:47:39] Model description:
Final prediction for new objects (level 0) = 
	 0.66234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.33766 * (5 averaged models Lvl_0_Pipe_1_Mod_0_Tuned_LightGBM) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.66234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.33766 * (5 averaged models Lvl_0_Pipe_1_Mod_0_Tuned_LightGBM) 



[LAMA FS+LGBM/Linear] ROC-AUC: 0.7217  F1: 0.4082


#### Выбор лучшего

In [49]:
results = {}
results['LAMA_DEFAULT'] = (auc1, f1_1)
results['LAMA_FAST'] = (auc2, f1_2)
results['LAMA_ADVANCED'] = (auc3, f1_3)
results['LAMA_FS'] = (auc4, f1_4)
for k, v in results.items():
    print(f'{k:15}| ROC-AUC: {v[0]:.4f} | F1: {v[1]:.4f}')

best_lama_name = max(results, key=lambda k: results[k][0])
print(f'--> Лучший бэйзлайн: {best_lama_name}, ROC-AUC = {results[best_lama_name][0]:.4f}')

LAMA_DEFAULT   | ROC-AUC: 0.7217 | F1: 0.6557
LAMA_FAST      | ROC-AUC: 0.7138 | F1: 0.6552
LAMA_ADVANCED  | ROC-AUC: 0.5000 | F1: 0.0000
LAMA_FS        | ROC-AUC: 0.7217 | F1: 0.4082
--> Лучший бэйзлайн: LAMA_DEFAULT, ROC-AUC = 0.7217


Классическая конфигурация LAMA, несмотря на консерватизм, дала лучший результат среди тестируемых.

### Собственное решение 1

In [50]:
# PIPELINE 1: LightGBM + scale + auto feature selection

# PIPELINE 1A: только scaling + LGBM (без фичер селекции)
pipe_lgbm_base = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_base.fit(X_train, y_train)
pred_lgbm_base = pipe_lgbm_base.predict_proba(X_test)[:, 1]
auc_lgbm_base = roc_auc_score(y_test, pred_lgbm_base)
print(f'[Pipeline LGBM basic] ROC-AUC: {auc_lgbm_base:.4f}')

# PIPELINE 1B: scaling + feature selection + LGBM
# Селектор тренируется внутри пайплайна по cross-val на train (без data leakage)
pipe_lgbm_fs = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LGBMClassifier(n_estimators=100, random_state=42), threshold="median")),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_fs.fit(X_train, y_train)
pred_lgbm_fs = pipe_lgbm_fs.predict_proba(X_test)[:, 1]
auc_lgbm_fs = roc_auc_score(y_test, pred_lgbm_fs)
print(f'[Pipeline LGBM + feature selection] ROC-AUC: {auc_lgbm_fs:.4f}')

[Pipeline LGBM basic] ROC-AUC: 0.6980
[Pipeline LGBM + feature selection] ROC-AUC: 0.6742


### Собственное решение 2

In [51]:
# PIPELINE 2: CatBoost + простая обработка пропусков
pipe_catboost = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('catboost', CatBoostClassifier(iterations=200,
                                    learning_rate=0.05,
                                    eval_metric='AUC',
                                    random_state=42,
                                    verbose=0))
])

pipe_catboost.fit(X_train, y_train)
pred_cat = pipe_catboost.predict_proba(X_test)[:, 1]
auc_cat = roc_auc_score(y_test, pred_cat)
print(f'[Pipeline CatBoost] ROC-AUC: {auc_cat:.4f}')


[Pipeline CatBoost] ROC-AUC: 0.7048


### Собственное решение 3

In [52]:
# PIPELINE 3: Logistic Regression + StandardScaler + гиперпараметры GridSearch
pipe_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=300))
])

param_grid = {'logreg__C': [0.01, 0.1, 1, 3, 10]}
grid = GridSearchCV(pipe_logreg, param_grid, scoring='roc_auc', cv=5)
grid.fit(X_train, y_train)

print(f"[Pipeline LogReg] Лучшее C: {grid.best_params_}")
best_logreg = grid.best_estimator_
preds_lr = best_logreg.predict_proba(X_test)[:, 1]
auc_lr = roc_auc_score(y_test, preds_lr)
print(f'[Pipeline LogReg tuned] ROC-AUC: {auc_lr:.4f}')

[Pipeline LogReg] Лучшее C: {'logreg__C': 0.1}
[Pipeline LogReg tuned] ROC-AUC: 0.7455


### Вывод
Свое решение оказалось лучше LAMA baseline: максимум ROC-AUC своего решения: 0.7455 (пайплайн 3), лучшего baseline: 0.7217


----
### [Дополнительно]

1) Проблема 1 - не GridSearch "в лоб", а хотя бы RandomSearch.

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from scipy.stats import uniform, randint

pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('pca', PCA(n_components=0.95)),
    ('lgbm', LGBMClassifier(verbose=-1, random_state=42))
])

param_dist = {
    'lgbm__n_estimators': randint(100, 500),
    'lgbm__learning_rate': uniform(0.01, 0.1),
    'lgbm__max_depth': randint(3, 7),
    'lgbm__num_leaves': randint(5, 20),
    'lgbm__subsample': uniform(0.6, 0.4),
    'lgbm__reg_alpha': uniform(0, 5),
    'lgbm__reg_lambda': uniform(0, 5)
}

rs = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',
    cv=5
    # random_state=42,
    # n_jobs=-1
)

rs.fit(X_train, y_train)
print(f"Best params: {rs.best_params_}")
best_model = rs.best_estimator_
preds_tuned = best_model.predict_proba(X_test)[:, 1]
auc_tuned = roc_auc_score(y_test, preds_tuned)
print(f'[Pipeline LGBM + RandomSearch] ROC-AUC: {auc_tuned:.4f}')

Best params: {'lgbm__learning_rate': np.float64(0.021586905952512975), 'lgbm__max_depth': 5, 'lgbm__n_estimators': 300, 'lgbm__num_leaves': 12, 'lgbm__reg_alpha': np.float64(2.247253370691017), 'lgbm__reg_lambda': np.float64(0.47705058245205656), 'lgbm__subsample': np.float64(0.7483273008793065)}
[Pipeline LGBM + RandomSearch] ROC-AUC: 0.6810


**Вывод:** на решении (#3), где ранее использовался GridSearch, те же метрики были 0.7455 - после использования RandomSearch стало хуже.

2) Проблема 2 - пофильтерить все-таки коррелирующие фичи

In [55]:
def remove_highcorr_features(x, threshold):
    corr_matrix = x.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f"Dropping {len(to_drop)} correlated features: {to_drop}")
    return x.drop(columns=to_drop), to_drop

X_train_uncorr, dropped_cols = remove_highcorr_features(X_train, 0.80)
X_test_uncorr = X_test.drop(columns=dropped_cols)

print(f"Cleaned train shape: {X_train_uncorr.shape}")

pipe_uncorr = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=300, C=1.0))
])

pipe_uncorr.fit(X_train_uncorr, y_train)
pred_uncorr = pipe_uncorr.predict_proba(X_test_uncorr)[:, 1]
auc_uncorr = roc_auc_score(y_test, pred_uncorr)
print(f'\n[Alternative solution: NO correlation] ROC-AUC: {auc_uncorr:.4f}')
print("-" * 10)


Dropping 4 correlated features: ['f4_p300', 'p4_p300', 'p8_p300', 'o2_p300']
Cleaned train shape: (120, 15)

[Alternative solution: NO correlation] ROC-AUC: 0.7387
----------


**Вывод:** после удаления признаков с коррелацией более 80% ROC-AUC опустился с 0.7455 до 0.7387 при том же решении.

3) Проблема 3 - странности результатов я думаю связаны в первую очередь с тем, что данных очень мало, медицинские данные редки и как правило довольно шумны.

In [56]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

# лимитим max_depth и num_leaves
# добавила PCA - полезно для EEG и заодно полечит корреляционность

pipe_lgbm_constrained = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),
    ('lgbm', LGBMClassifier(
        n_estimators=200,
        learning_rate=0.03,   # помедленнее обучение
        num_leaves=7,         # малое количество
        max_depth=3,          # низкая граница
        reg_alpha=1,
        reg_lambda=1,
        random_state=42,
        verbose=-1
    ))
])

pipe_lgbm_constrained.fit(X_train, y_train)
pred_lgbm_fix = pipe_lgbm_constrained.predict_proba(X_test)[:, 1]
auc_lgbm_fix = roc_auc_score(y_test, pred_lgbm_fix)
print(f'[Pipeline LGBM Constrained + PCA] ROC-AUC: {auc_lgbm_fix:.4f}')

[Pipeline LGBM Constrained + PCA] ROC-AUC: 0.6867


In [57]:
# Пробуем еще воутинг:
clf1 = LogisticRegression(C=1, solver='liblinear', random_state=42)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf3 = SVC(probability=True, kernel='rbf', C=1, random_state=42)

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('svm', clf3)],
    voting='soft'
)

pipe_voting = Pipeline([
    ('scaler', StandardScaler()),
    ('voting', eclf)
])

pipe_voting.fit(X_train, y_train)
pred_vote = pipe_voting.predict_proba(X_test)[:, 1]
auc_vote = roc_auc_score(y_test, pred_vote)
print(f'[Pipeline Voting Ensemble] ROC-AUC: {auc_vote:.4f}')


[Pipeline Voting Ensemble] ROC-AUC: 0.7115
